# Financial Risk Analysis FastAPI API

This notebook contains the code and documentation for a FastAPI app that uses a fine-tuned causal language model to perform financial risk analysis from input text.

## 1. Import Libraries

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
import os

## 2. Setup Model Directory and Load Model & Tokenizer

In [ ]:
# Resolve the absolute directory of the script and model path
script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
model_path = os.path.join(script_dir, "mistral-combined-finetuned-weights")

# Load tokenizer and model locally from the specified folder
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    local_files_only=True
)
model.eval()

## 3. Initialize FastAPI App and Define Request Schema

In [ ]:
# Initialize FastAPI app instance with a title
app = FastAPI(title="Financial Risk Analysis API")

# Define the request schema using Pydantic for input validation
class AnalysisRequest(BaseModel):
    input_text: str                # Text to analyze for financial risk
    max_new_tokens: int = 512     # Max tokens to generate (optional, default 512)
    temperature: float = 0.7       # Sampling temperature for generation (optional)
    top_p: float = 0.95            # Nucleus sampling probability (optional)


## 4. Define Inference Function

In [ ]:
def generate_analysis(input_data: str, max_new_tokens=512, temperature=0.7, top_p=0.95):
    prompt = (
        "You are an expert financial risk analyst. Analyze the provided text for financial risks, "
        "and output a structured assessment in JSON format including risk detection, specific risk flags, "
        "financial exposure details, and analysis notes. "
        f"{input_data}"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        start = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        end = time.time()

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n⏱️ Generation time: {end - start:.2f} seconds")
    return result


## 5. Define API Endpoint

In [ ]:
@app.post("/analyze")
def analyze_risk(request: AnalysisRequest):
    output = generate_analysis(
        input_data=request.input_text,
        max_new_tokens=request.max_new_tokens,
        temperature=request.temperature,
        top_p=request.top_p
    )
    return {"analysis": output}


## 6. Usage Instructions

### Start the server (run in terminal):
```bash
uvicorn Daniel Gan.main:app --reload --host 0.0.0.0 --port 8000
```

### Make a POST request (example with curl):
```bash
curl -X POST "http://localhost:8000/analyze" -H "Content-Type: application/json" -d '{
  "input_text": "The company faces liquidity risks due to recent market volatility.",
  "max_new_tokens": 256,
  "temperature": 0.7,
  "top_p": 0.9
}'
```

### Or using Python requests:
```python
import requests
url = "http://localhost:8000/analyze"
payload = {
    "input_text": "The company faces liquidity risks due to recent market volatility.",
    "max_new_tokens": 256,
    "temperature": 0.7,
    "top_p": 0.9
}
response = requests.post(url, json=payload)
print(response.json()["analysis"])
```

### Notes:
- Ensure the model files are available locally in `mistral-combined-finetuned-weights` next to your script.
- Adjust generation parameters to control the output.
- The output is a JSON-formatted string describing detected financial risks.
